In [1]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import numpy as np
#   If you are getting ModuleNotFoundError, uncomment the following line...
import sys
#   ...and then replace 'your_path' with your path to this CMOR492-DWS directory
#       (if you're pulling from the github, the directory is called CMOR492-DWS as of 19/02/2025)
# sys.path.append('your_path/CMOR492-DWS')
# sys.path.append('/Users/danielsuarez/Documents/Academic/Spring2025/SeniorDesign/CMOR492-DWS/')
# sys.path.append("C:\\Users\\gabri\\Documents\\CMOR492-DWS") # This is the path on Gabriel Lundquist's machine.
sys.path.append("C:\\Users\\jacob\\OneDrive\\Documents\\College\\Rice University\\CMOR492-DWS") # This is the path on Jacob's machine.
from network_construction.network import source_treatment, get_Utown


In [7]:
G = get_Utown()
source_nodes, treatment_nodes = source_treatment(G, False) #, 40)  # <-- Specify # starting points for treatment node algorithm

In [8]:
### MODEL PARAMETERS

# TODO: What if we just make pipe size continuous/linear

Path = {}  # Set of shortest paths from each source node i to each treatment node j
NLinks = {}  # Number of edges in each path
L = {}  # Length of each path (distance)

for i in source_nodes:
    for j in treatment_nodes:
        path = nx.shortest_path(G, source=i, target=j, weight='length')
        Path[i, j] = path
        NLinks[i, j] = len(path)-1
        L[i, j] = nx.path_weight(G, path, weight='length')
LE = {e: G.edges[e]['length'] for e in G.edges}  # Length of edge e
EL = {v: G.nodes[v]['elevation'] for v in G.nodes}  # Elevation of node v


D = [0.2, 0.25, 0.3, 0.35, 0.40, 0.45]  # Pipe diameters
CP = {0.05: 8.7, 0.06: 9.5, 0.08: 11,
                       0.1: 12.6, 0.15: 43.5, 0.2: 141,
                       0.25: 151, 0.3: 161, 0.35: 180,
                       0.4: 190, 0.45: 200}  # Cost per unit of pipe


SR = {}  # Production at Source node i
CAP = {}  # Capacity at treatment node j

for node in source_nodes:
    G.nodes[node]['production'] = .17
    SR[node] = .17

total_flow = sum(SR.values())

for node in treatment_nodes:
    G.nodes[node]['capacity'] = 1000
    CAP[node] = 1000

Vmin = 0.6 * 60
Vmax = 3 * 60

CE = 25  # Cost of Excavation
CB = 6  # Cost of Bedding
TR = 44000  # Fixed Cost of Treatment Plant
TRFlow = 100  # Variable Cost of Treatment
PICost = 30

PF = {'0.05': 8.7, '0.06': 9.5, '0.08': 11,
                       '0.1': 12.6, '0.15': 43.5, '0.2': 141,
                       '0.25': 151, '0.3': 161, '0.35': 180,
                       '0.4': 190, '0.45': 200}  # Fixed Cost of Piping

CT = 1000000000  # Cost of trucking
M = 1e6

Smin = 0.01
Smax = 0.1
W = 0.5  # Buffer Width

R = 0  # Discount factor

In [9]:
m = gp.Model()

### DECISION VARIABLES

x = m.addVars(Path.keys(), vtype=GRB.BINARY, name='x')  # Path ij used 
y = m.addVars(treatment_nodes, vtype=GRB.BINARY, name='y')  # treatment at node j 
z = m.addVars(G.edges, vtype=GRB.BINARY, name='z')  # edge e used



# v This v is added in ### ADD DECISION VARIABLES FOR MULTIPERIOD
# d = m.addVars(G.edges, D, vtype=GRB.BINARY, name='d')  # New replacement pipe size s at edge e 

a = m.addVars(G.edges, D, vtype=GRB.BINARY, name='a')  # Pipe size s at edge e 

# Node Elevation
el = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, name='el')  # Elevation at node v 

# Recourse Amount
r = m.addVars(source_nodes, vtype=GRB.CONTINUOUS, lb=0.0, name='r')  # Flow handled by trucking at edge e 

# Flow in Edge e
Q = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb=0.0, name='Q')  # Flow in Edge e 

# Path Flow
p = m.addVars(Path.keys(), vtype=GRB.CONTINUOUS, lb = 0.0, name='p')  # Flow through path ij 

m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-08


In [10]:
### CONSTRAINTS (single period; see below for multiperiod)

# NODE PRODUCTION MINUS RECOURSE
node_prod_rec = m.addConstrs((p[i, j] >= (SR[i] * x[i, j]) - r[i] for i, j in Path.keys()), name='node_prod_rec')

# TREATMENT CAPACITY
treat_cap = m.addConstrs((gp.quicksum(p[i, j] for i in source_nodes) <= CAP[j] * y[j] for j in treatment_nodes), name='treat_cap')

#  NODE ASSIGNMENT
node_assign = m.addConstrs((gp.quicksum(x[i, j] for j in treatment_nodes) == 1 for i in source_nodes), name='node_assign')

# PIPE SIZING
pipe_sizing = m.addConstrs((gp.quicksum(a[*e, s] for s in D) == z[e] for e in G.edges), name='pipe_sizing')  # ALWAYS BE SURE TO UNPACK e

# TODO: Go through this with John
# FLOW DEFINITION
def is_sublist(short_list, long_list):
    for i in range(len(long_list) - len(short_list) + 1):
        if long_list[i:i + len(short_list)] == short_list:
            return True
    return False

flow_def = m.addConstrs((Q[e] == gp.quicksum(p[i, j] for i, j in Path.keys() if is_sublist(list((e[0], e[1])),Path[i,j])) for e in G.edges), name='flow_def')


# MIN/MAX SLOPE
min_slope = m.addConstrs((el[e[0]] - el[e[1]] >= (LE[e] * Smin) - (M * (1 - z[e])) for e in G.edges), name='min_slope')
max_slope = m.addConstrs((el[e[0]] - el[e[1]] <= (LE[e] * Smax) + (M * (1 - z[e])) for e in G.edges), name='max_slope')

# FLOW VELOCITY LIMIT
flow_vel = m.addConstrs((Q[e] <= Vmax * gp.quicksum((np.pi / 8) * (s**2) * (a[*e, s]) for s in D) for e in G.edges), name='flow_vel')

# PIPES UNDERGROUND
underground = m.addConstrs((el[u] <= EL[u] for u in G.nodes), name='underground')
m.update()
# EDGE ACTIVATION

# TODO: Go through this with John 2
# EDGE ACTIVATION
ePath = {}  # Use this for Edge Activation Constraint
for e, p_ in Path.items(): # Using temporary variable p_ since p is already a decision variable (see above)
    ePath[e] = [(p_[l - 1], p_[l]) for l in range(1, len(p_))]

edge_activate = m.addConstrs((gp.quicksum(z[e] for e in ePath[i, j]) >= NLinks[i, j] * x[i, j] for i, j in Path), name='edge_activate')


# ENVELOPES FOR MANNING

T = 11.9879
P = lambda LE, s: LE / (T * (s**(16/3)))
Qmax = lambda s: Vmax * ((np.pi / 8) * (s**2))


alpha = m.addVars(G.edges, D, lb=0, name='alpha')
beta = m.addVars(G.edges, D, lb=0, name='beta')


alpha_2 = m.addConstrs((alpha[*e, s] >= Q[e] + a[*e, s] * Qmax(s) - ( Qmax(s)) for e in G.edges for s in D), name='alpha_2')
alpha_3 = m.addConstrs((alpha[*e, s] <= Qmax(s) * a[*e, s] for e in G.edges for s in D), name='alpha_3')
alpha_4 = m.addConstrs((alpha[*e, s] <= Q[e] for e in G.edges for s in D), name='alpha_4')
alpha_5 = m.addConstrs((alpha[*e, s] <= Qmax(s) for e in G.edges for s in D), name='alpha_5')

beta_2 = m.addConstrs((beta[*e, s] >= (Qmax(s) * Q[e]) + (Qmax(s) * alpha[*e, s]) - (Qmax(s)**2) for e in G.edges for s in D), name='beta_2')
beta_3 = m.addConstrs((beta[*e, s] <= Qmax(s) * alpha[*e, s] for e in G.edges for s in D), name='beta_3')
beta_4 = m.addConstrs((beta[*e, s] <= Qmax(s) * Q[e] for e in G.edges for s in D), name='beta_4')

# manning_2 = m.addConstrs((el[e[1]] - el[e[0]] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= 0 for e in G.edges), name='manning_2')

# ADDED THE BIG M THING HERE BUT IDK IF IT COULD BE IMPROVED
manning_2 = m.addConstrs((el[e[1]] - el[e[0]] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= (1-z[e])*M for e in G.edges), name='manning_2')
m.update()

In [11]:
# OBJECTIVE EPXR 1: TREATMENT COSTS
treat_cost = gp.LinExpr()
for j in treatment_nodes:
    treat_cost.addTerms(TR, y[j])
    for i in source_nodes:
        treat_cost.addTerms(TRFlow * SR[i], x[i, j])

# OBJECTIVE EXPR 2: EXCAVATION COSTS
excav_cost_f = lambda u, v: gp.QuadExpr(CE * (((EL[u] - el[u]) + (EL[v] - el[v])) / 2) * LE[u, v] * gp.quicksum(s + ((2*W) * a[u, v, s]) for s in D))

# OBJECTIVE EXPR 3: BEDDING COSTS
bed_cost_f = lambda u, v: gp.LinExpr(CB * LE[u, v] * gp.quicksum(s + ((2*W) * a[u, v, s]) for s in D))
# OBJECTIVE EXPR 4: PIPE COSTS
pipe_cost_f = lambda u, v: gp.LinExpr(LE[u, v] * gp.quicksum(CP[s] * a[u, v, s] for s in D))

excav_bed_cost = gp.quicksum(excav_cost_f(u, v) + bed_cost_f(u, v) + pipe_cost_f(u, v) for u, v in G.edges)

# OBJECTIVE EXPR 5: RECOURSE TRUCKING

rec_cost = gp.LinExpr()
for i in source_nodes:
    rec_cost.addTerms(CT, r[i])

m.setObjective(treat_cost + excav_bed_cost + rec_cost, GRB.MINIMIZE)

# m.setObjective(0, GRB.MINIMIZE)

m.update()
print(f"Model has {m.NumVars} variables and {m.NumConstrs} constraints.")

Model has 24786 variables and 38926 constraints.


In [12]:
# m.write("singleperiod_nocontext2.lp")
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 38926 rows, 24786 columns and 286316 nonzeros
Model fingerprint: 0x4f3accb2
Model has 6054 quadratic objective terms
Variable types: 14060 continuous, 10726 integer (10726 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+06]
  Objective range  [2e+01, 1e+09]
  QObjective range [3e+02, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 16481 rows and 4597 columns
Presolve time: 2.67s
Presolved: 28175 rows, 25919 columns, 262824 nonzeros
Variable types: 15409 continuous, 10510 integer (10510 binary)
Found heuristic solution: objective 2.694206e+09

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing primal log only...

Root relaxa

##### Here begins multiperiod work

In [18]:
### Initializing multiperiod-related variables
n_periods = 3
periods = list(range(n_periods)) # `periods` can be an arbitrary list/tuple, here we just make it (0,1,2)
# v this v is shorthand for creating historical dictionaries for each decision variable (wrap this in function?)
(xt, yt, zt, at, elt, rt, Qt, pt) = [{period: None for period in periods} for _ in (x,y,z,a,el,r,Q,p)] 

##### Problems with modifying and adding constraints: 
1. When you add or modify a constraint, on model update it clears out the values of the variables, which causes the values of the historical dictionaries to change, if they're just copies of the `tupledict` returned by the constraint initialization functions.
2. Setting the `"rhs"` attribute on constraints requires a `double` argument, not an argument with `Gurobi.var` variables. 

##### Solution: 
1. Manually perform a deeper copy of the `tupledict`, storing keys and values rather than `Gurobi.var` variables
2. Manually remove and re-add the modified constraints

In [ ]:
### ONLY RUN FOR AN EXAMPLE OF MODEL UPDATING CLEARING THE VALUES OF THE GUROBI VARIABLES
print(f"{x = }")
d = m.addVars(G.edges, D, vtype=GRB.BINARY, name='d')
m.update()
print(f"{x = }")

In [19]:
# Write the value of each decision variable in the first period into the historical dictionaries
# Can't just call `x.copy()` for example, since that's a shallow copy so the values in the historical dict will change
def record_history(period_to_record):
    for gurobi_var_dict, history_dict in ((x,xt), (y,yt), (z,zt), (a,at), (el,elt), (r,rt), (Q,Qt), (p,pt)):
        history_dict[period_to_record] = {key: gurobi_var.X for key, gurobi_var in gurobi_var_dict.items()}

record_history(periods[0])

In [37]:
### CHANGE WASTEWATER FLOW
for node in source_nodes:
    # Arbitrary choice of new wastewater flow for testing purposes
    SR[node] = .34
    # G.nodes[node]['production'] = .34 # This variable isn't used anywhere else

In [20]:
### ADD DECISION VARIABLES AND CONSTRAINTS FOR MULTIPERIOD

current_period_index = 1

## New variables 
# What new size pipe needs to be installed, if any
d = m.addVars(G.edges, D, vtype=GRB.BINARY, name='d')

# Whether elevation at vertex v changes
c = m.addVars(G.nodes, vtype=GRB.BINARY, name='c') 


## New, non-time-dependent constraints
# CHANGE IN PIPE SIZE (ensures that at most 1 new pipe size can be selected)
pipe_size_change = m.addConstrs((gp.quicksum(d[*e, s] for s in D) <= 1 for e in G.edges), name='pipe_size_change')  
# Always unpack e, because the Gurobi tupledicts are indexed by scalar variables, not tuples. 
# You need [e0, e1, s], not [(e0,e1),s]

# VERTEX ELEVATION CHANGE ASSIGNMENT
elevation_change_assignment = m.addConstrs(
    (gp.quicksum(d[*e, s] for s in D) >= 0.5 * (c[e[0]] + c[e[1]]) + (z[*e] - 1) for e in G.edges), name='elevation_change_assignment')

# a-CONSTRAINT (ensures that a truly represents the accurate pipe size)
a_constraint = m.addConstrs(
    (a[*e, s] <= (d[*e, s] + at[periods[current_period_index-1]][*e, s]) for e in G.edges for s in D), name='a_constraint')
# We have to remove ^ this ^ constraint and re-add it after every period, because the numbers from the previous period will change

# VERTEX ELEVATION CHANGE ENFORCEMENT
max_elevation_change_enforcement = m.addConstrs(
    (c[u] >= (el[u] - elt[periods[current_period_index-1]][u]) / M for u in G.nodes), name='max_elevation_change_enforcement')
# We have to remove ^ this ^ constraint and re-add it after every period, because the numbers from the previous period will change
min_elevation_change_enforcement = m.addConstrs(
    (c[u] >= (elt[periods[current_period_index-1]][u] - el[u]) / M for u in G.nodes), name='min_elevation_change_enforcement')
# We have to remove ^ this ^ constraint and re-add it after every period, because the numbers from the previous period will change

# TREATMENT PLANT CONTINUITY
treatment_plant_continuity = m.addConstrs((y[j] >= yt[periods[current_period_index-1]][j] for j in treatment_nodes), name='treatment_plant_continuity')
# We have to remove ^ this ^ constraint and re-add it after every period, because the numbers from the previous period will change

m.update()

In [21]:
### EDIT CONSTRAINTS BEFORE STARTING NEW PERIOD
# REMOVES PREVIOUS CONSTRAINTS; YOU'LL FIND HANDLES TO THE NEW ONES RETURNED BY THIS METHOD
# We *could* just use the global keyword... but I'd like this function to be at least a little portable.
# It does still rely on lots of global variables
def edit_constrs_SR_change(current_period_index):
    # Because the "rhs" attribute on constraints requires a double value, not a gurobi.Var, 
    # we have to manually remove and re-add the constraints. 

    # NODE PRODUCTION MINUS RECOURSE (depends on modified wastewater flow SR)
    m.remove(node_prod_rec)
    # The _new emphasizes that this is a temporary, local variable that needs to be returned
    node_prod_rec_new = m.addConstrs((p[i, j] >= (SR[i] * x[i, j]) - r[i] for i, j in Path.keys()), name='node_prod_rec')

    # a-CONSTRAINT (depends on `a` from previous period)
    m.remove(a_constraint)
    a_constraint_new = m.addConstrs(
        (a[*e, s] <= (d[*e, s] + at[periods[current_period_index-1]][*e, s]) for e in G.edges for s in D), name='a_constraint')
    
    # VERTEX ELEVATION CHANGE (depends on elevation at previous period)
    m.remove(max_elevation_change_enforcement)
    m.remove(min_elevation_change_enforcement)
    max_elevation_change_enforcement_new = m.addConstrs(
        (c[u] >= (el[u] - elt[periods[current_period_index-1]][u]) / M for u in G.nodes), name='max_elevation_change_enforcement')
    min_elevation_change_enforcement_new = m.addConstrs(
        (c[u] >= (elt[periods[current_period_index-1]][u] - el[u]) / M for u in G.nodes), name='min_elevation_change_enforcement')

    # TREATMENT PLANT CONTINUITY (depends on treatment plant existence in previous period)
    m.remove(treatment_plant_continuity)
    treatment_plant_continuity_new = m.addConstrs(
        (y[j] >= yt[periods[current_period_index-1]][j] for j in treatment_nodes), name='treatment_plant_continuity')

    return (node_prod_rec_new, a_constraint_new, max_elevation_change_enforcement_new, min_elevation_change_enforcement_new, 
            treatment_plant_continuity_new)

(node_prod_rec, a_constraint, max_elevation_change_enforcement, min_elevation_change_enforcement, 
 treatment_plant_continuity) = edit_constrs_SR_change(current_period_index)
print(f"{current_period_index = }")

m.update()

current_period_index = 1


In [22]:
### CHANGE OBJECTIVE FUNCTION FOR MULTIPERIOD

# OBJECTIVE EPXR 1: add consideration of previous y_j to treatment costs
# Treatment cost term in objective function depends on wastewater flow
# Easiest to just reinitialize treat_cost rather than combing out the terms depending on previous SR
def edit_treatment_cost(current_period_index):
    treat_cost_new = gp.LinExpr()
    for j in treatment_nodes:
        treat_cost_new.add(y[j], TR)
        treat_cost_new.addConstant(-TR * yt[periods[current_period_index-1]][j])
        for i in source_nodes:
            treat_cost_new.add(x[i, j], TRFlow * SR[i])
    return treat_cost_new

    # All other objective expressions stay the same

treat_cost = edit_treatment_cost(current_period_index)

# Modify the objective to account for the new treatment cost and discount factor
m.setObjective((treat_cost + excav_bed_cost + rec_cost) / (1 + R)**periods[current_period_index], GRB.MINIMIZE)

In [23]:
# m.setObjective(0, GRB.MINIMIZE)
m.update()

In [24]:
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 43411 rows, 28015 columns and 300793 nonzeros
Model fingerprint: 0x49751beb
Model has 6054 quadratic objective terms
Variable types: 14060 continuous, 13955 integer (13955 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+06]
  Objective range  [2e+01, 1e+09]
  QObjective range [3e+02, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-05, 1e+06]

MIP start from previous solve produced solution with objective 8.44856e+06 (0.29s)
Loaded MIP start from previous solve with objective 8.44856e+06

Presolve removed 20994 rows and 7854 columns
Presolve time: 2.68s
Presolved: 28147 rows, 25891 columns, 262070 nonzeros
Variable types: 15409 continuous, 10482 integer (10482 binary)

Deterministic concurrent L

In [ ]:
for v in r:
    if r[v].X > 0:
        print(r[v].VarName, r[v].X)

In [ ]:
m.computeIIS()

for c in m.getConstrs():
    if c.IISConstr:
        print(f"Constraint {c.ConstrName} is in the IIS")

for v in m.getVars():
    if v.IISLB > 0:
        print(f"Lower bound of {v.VarName} is in the IIS")
    elif v.IISUB > 0:
        print(f"Upper bound of {v.VarName} is in the IIS")


In [25]:
x_0 = {str(xv) : x[xv].X for xv in x}
y_0 = {yv : y[yv].X for yv in y}
z_0 = {str(zv) : z[zv].X for zv in z}

d_0 = {str(dv) : a[dv].X for dv in a}
el_0 = {elv: el[elv].X for elv in el}

In [26]:
import json

In [27]:
with open("x_sol_later.json", "w") as f:
    json.dump(x_0, f)

In [28]:
with open("z_sol_later.json", "w") as f:
    json.dump(z_0, f)

with open("d_sol_later.json", "w") as f:
    json.dump(d_0, f)

with open("el_sol_later.json", "w") as f:
    json.dump(el_0, f)